In [1]:
import numpy as np

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0'

import tensorflow.keras.backend as K
def customLoss(yTrue,yPred):
    print("yTrue shape\n")
    print(yTrue.shape)
    print("yTrue value\n")
    print(yTrue)
    print("yPred shape\n")
    print(yPred.shape)
    print("yPred value\n")
    print(yPred)
    a = 0
    if type(yTrue.shape[0]) != int:
        return 0
    for i in range(yTrue.shape[0]):
        a += -(np.log(yPred[i][yTrue[i]]))
    a = a/yTrue.shape[0]
    return a

In [3]:
import tensorflow.keras.backend as K
def customLoss(yTrue,yPred):
    print("yTrue.shape")
    print(yTrue)
    print('\n')
    print("yPred.shape")
    print(yPred)
    print('\n')
    if yTrue.shape[0] == None:
        return 1e-7
    temp = []
    yPred_unpack = tf.unstack(yPred,axis = 1)
    print("I am here\n")
    for num,a in enumerate(yPred_unpack):
        print("I'm inside loop\n")
        print("This is the num:"+str(num)+'\n')
        print("This is a\n")
        print(a)
        print('\n')
        b = K.softmax(a)
        print("this is b\n")
        print(b)
        print('\n')
        print('this is ytrue[num]')
        print(yTrue[0,num])
        print('\n')
        temp.append(K.categorical_crossentropy(yTrue[0,num],b))

    print("I'm outside the loop")
    temp_tensor = tf.concat(temp,0)
    print("all the temp values\n")
    print(temp_tensor)
    print('\n')
    mean = tf.reduce_mean(temp_tensor)
    print("reduced_mean values\n")
    print(mean)
    print('\n')
    return mean

In [4]:
def create_model_all_conv():
 
    model = tf.keras.models.Sequential()
#convolutional layer with rectified linear unit activation
    model.add(keras.layers.Conv1D(128, kernel_size=1,padding = "Same",
                 activation=tf.keras.activations.relu,
                 input_shape=(None,26)))
#32 convolution filters used each of size 5
#again
    model.add(keras.layers.Conv1D(64, kernel_size=4, padding = "Same", activation=tf.keras.activations.relu))

    model.add(keras.layers.Conv1D(32, kernel_size=4,padding = "Same", activation=tf.keras.activations.relu))    
    model.add(keras.layers.Conv1D(16, kernel_size=4,padding = "Same", activation=tf.keras.activations.relu)) 
    model.add(keras.layers.Conv1D(8, kernel_size=4,padding = "Same", activation=tf.keras.activations.relu)) 
    model.add(keras.layers.Conv1D(4, kernel_size=4,padding = "Same")) 

    model.compile(optimizer=tf.keras.optimizers.Adadelta(),
                loss=customLoss,
                )

    return model

def input_generator(input_feature, input_label):

    for i in range(input_label.shape[0]):
        shape = list(input_feature[i].shape)
        shape.insert(0,-1)
        yield input_feature[i].reshape(shape), input_label[i].reshape(-1,input_label[i].shape[0])

In [5]:
train_feature_file_format = "/home/qingyliu/test/training_data/train_feature_0.npy"

train_label_file_format = "/home/qingyliu/test/training_data/train_label_0.npy"

train_features= np.load(train_feature_file_format,allow_pickle=True)
train_labels = np.load(train_label_file_format,allow_pickle=True)


In [42]:
train_features[0].shape

(141, 26)

In [6]:
checkpoint_path_all_conv = "/home/qingyliu/test/training_checkpoint/cp_test.ckpt"
checkpoint_dir_all_conv = os.path.dirname(checkpoint_path_all_conv)
cp_callback_all_conv = tf.keras.callbacks.ModelCheckpoint(checkpoint_path_all_conv,
                                                 save_weights_only=True,
                                                 verbose=1)
model_all_conv = create_model_all_conv()



model_all_conv.fit(train_features, train_labels,  epochs = 500, batch_size=5,
                callbacks = [cp_callback_all_conv])

yTrue.shape
Tensor("conv1d_5_target:0", shape=(None, None, None), dtype=float32)


yPred.shape
Tensor("conv1d_5/Identity:0", shape=(None, None, 4), dtype=float32)




ValueError: Error when checking input: expected conv1d_input to have 3 dimensions, but got array with shape (13, 1)